# Initial Zillow Clustering Project Notebook 

In [2]:
import pandas as pd
import numpy as np

# Vis tools
import matplotlib.pyplot as plt
import seaborn as sns

# Custom Modules

# stats and modeling tools
from sklearn.cluster import KMeans
import scipy.stats as stats
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

# supress scientific notation
np.set_printoptions(suppress=True)

# silence warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# set and use zillow color palette
zpalette = ['#1277e1', '#f3ad35', '#0b449c', '#5289e4', '#c3eafb']
sns.set_palette(zpalette)
sns.color_palette()

[(0.07058823529411765, 0.4666666666666667, 0.8823529411764706),
 (0.9529411764705882, 0.6784313725490196, 0.20784313725490197),
 (0.043137254901960784, 0.26666666666666666, 0.611764705882353),
 (0.3215686274509804, 0.5372549019607843, 0.8941176470588236),
 (0.7647058823529411, 0.9176470588235294, 0.984313725490196)]

<hr style="border-top: 10px groove #1277e1; margin-top: 1px; margin-bottom: 1px"></hr>

## Project Goals and Ideas
### The Big Question: What is driving the errors in the Zestimates?

For this project the target is `logerror`

#### Goal: Identify Drivers of Error and Create model(s) that account for those drivers.

#### Initial Thoughts
- After only looking at size (bedroom count, bathroom count and square footage for the house itself) model was not great
- Location is very important in real estate. As they say Location Location Location
    - Use Latitude and Longitude to narrow down groups
- Price per square foot is something lots of realtors use for comparing houses that aren't the same size 


In [9]:
# defining some functions to make it easier. will go in Wrangle function
from env import host, password, user
import os

###################### Getting database Url ################
def get_db_url(db_name, user=user, host=host, password=password):
    """
        This helper function takes as default the user host and password from the env file.
        You must input the database name. It returns the appropriate URL to use in connecting to a database.
    """
    url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'
    return url

######################### get generic data #########################
def get_any_data(database, sql_query):
    '''
    put in the query and the database and get the data you need in a dataframe
    '''

    return pd.read_sql(sql_query, get_db_url(database))

######################### get Zillow Data #########################
def get_zillow_data():
    '''
    This function reads in Zillow data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    sql_query = """
                SELECT parcelid, airconditioningtypeid, airconditioningdesc, architecturalstyletypeid, architecturalstyledesc,
                bathroomcnt, bedroomcnt, buildingclasstypeid, buildingclassdesc, buildingqualitytypeid,
                decktypeid, calculatedfinishedsquarefeet, fips, fireplacecnt, fireplaceflag, garagecarcnt, garagetotalsqft,
                hashottuborspa, latitude, longitude, lotsizesquarefeet, poolcnt, poolsizesum, propertycountylandusecode,
                propertylandusetypeid, propertylandusedesc, propertyzoningdesc, rawcensustractandblock, 
                regionidcity, regionidcounty, regionidneighborhood, roomcnt, threequarterbathnbr, typeconstructiontypeid, typeconstructiondesc, unitcnt, yearbuilt, numberofstories, structuretaxvaluedollarcnt, taxvaluedollarcnt, assessmentyear, 
                landtaxvaluedollarcnt, taxamount, censustractandblock, logerror, transactiondate 
                FROM properties_2017 AS p
                JOIN predictions_2017 USING (parcelid)
                INNER JOIN (SELECT parcelid, MAX(transactiondate) AS transactiondate
                FROM predictions_2017
                GROUP BY parcelid) 
                AS t USING (parcelid, transactiondate)
                LEFT JOIN airconditioningtype USING (airconditioningtypeid)
                LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
                LEFT JOIN buildingclasstype USING (buildingclasstypeid)
                LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
                LEFT JOIN propertylandusetype USING (propertylandusetypeid)
                LEFT JOIN storytype USING (storytypeid)
                LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
                WHERE latitude IS NOT NULL AND longitude IS NOT NULL 
                AND transactiondate LIKE "2017%%";
                """
    if os.path.isfile('zillow_data.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('zillow_data.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = pd.read_sql(sql_query, get_db_url('zillow'))
        
        # Cache data
        df.to_csv('zillow_data.csv')

    return df

In [10]:
def overview(df, thresh = 10):
    '''
    This function takes in a dataframe and prints out useful things about each column.
    Unique values, value counts for columns less than 10 (can be adjusted with optional arguement thresh)
    Whether or not the row has nulls
    '''
    # create list of columns
    col_list = df.columns
    
    # loop through column list
    for col in col_list:
        # seperator using column name
        print(f'============== {col} ==============')
        
        # print out unique values for each column
        print(f'# Unique Vals: {df[col].nunique()}')
        
        # if number of things is under or equal to the threshold  print a value counts
        if df[col].nunique() <= thresh:
            print(df[col].value_counts(dropna = False).sort_index(ascending = True))
            
        # if the number is less than 150 and not an object, bin it and do value counts
        elif (df[col].nunique() < 150) and df[col].dtype != 'object' :
            print(df[col].value_counts(bins = 10, dropna=False).sort_index(ascending = True))
        
        # Space for readability 
        print('')
       

In [11]:
# I saw this on the afore mentioned kaggle site. This is the credit that author gave.
# credit: https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction. 
# One of the best notebooks on getting started with a ML problem.

def missing_values_table(df):
    '''
    this function takes a dataframe as input and will output metrics for missing values, 
    and the percent of that column that has missing values
    '''
    # Total missing values
    mis_val = df.isnull().sum()
    
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    
    # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
        
        # Return the dataframe with missing information
    return mis_val_table_ren_columns

<hr style="border-top: 10px groove #1277e1; margin-top: 1px; margin-bottom: 1px"></hr>

## Get the Data and Overview
- Shape of data
- Number unique values
- Distribution of values
- Null Value exploration

In [12]:
# get zillow data with function above
df = get_zillow_data()

In [13]:
# have a total of 77380 rows 46 columns
df.shape

(77380, 46)

In [15]:
# see the names and types of all columns
# Lots of Nulls to deal with
# might need to do some splitting up vx object, float and int
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 46 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77380 non-null  int64  
 1   airconditioningtypeid         24953 non-null  float64
 2   airconditioningdesc           24953 non-null  object 
 3   architecturalstyletypeid      206 non-null    float64
 4   architecturalstyledesc        206 non-null    object 
 5   bathroomcnt                   77380 non-null  float64
 6   bedroomcnt                    77380 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingclassdesc             15 non-null     object 
 9   buildingqualitytypeid         49671 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  calculatedfinishedsquarefeet  77184 non-null  float64
 12  fips                          77380 non-null  float64
 13  f

In [16]:
overview(df)

============== parcelid ==============
# Unique Vals: 77380

============== airconditioningtypeid ==============
# Unique Vals: 5
1.0     23133
5.0       167
9.0        26
11.0       53
13.0     1574
NaN     52427
Name: airconditioningtypeid, dtype: int64

============== airconditioningdesc ==============
# Unique Vals: 5
Central          23133
None               167
Refrigeration       26
Wall Unit           53
Yes               1574
NaN              52427
Name: airconditioningdesc, dtype: int64

============== architecturalstyletypeid ==============
# Unique Vals: 5
2.0         5
3.0         3
7.0       172
8.0        19
21.0        7
NaN     77174
Name: architecturalstyletypeid, dtype: int64

============== architecturalstyledesc ==============
# Unique Vals: 5
Bungalow             5
Cape Cod             3
Contemporary       172
Conventional        19
Ranch/Rambler        7
NaN              77174
Name: architecturalstyledesc, dtype: int64

============== bathroomcnt ==============
#

#### Takeaways
- not sure what roomcnt is and how it's different from bedrooms and bathrooms 
- three quarter bath seems redundant 
- most of the homes here are single family 
- there's someone who has a 14 car garage (maybe it's a quadruplex or something)
- three houses have 5 fireplaces. Must be really cold there in Southern California
- Date could be changed to a datetime object

### Taking a look at Nulls

In [17]:
missing_values_table(df)

Your selected dataframe has 46 columns.
There are 31 columns that have missing values.


,Missing Values,% of Total Values
buildingclasstypeid,77365,100.0
buildingclassdesc,77365,100.0
fireplaceflag,77208,99.8
architecturalstyletypeid,77174,99.7
architecturalstyledesc,77174,99.7
typeconstructiondesc,77158,99.7
typeconstructiontypeid,77158,99.7
decktypeid,76766,99.2
poolsizesum,76513,98.9
hashottuborspa,75842,98.0


#### Takeaways from NaNs
- buildingclassdesc and buildingclasstypeid have too many missing, will drop
- fireplaceflag seems to not have been used as much as fireplacecnt (drop fireplaceflag)
- the architectural columns also don't have enough to go on, nothing to replace with
- typeconstructiondesc and typeconstructiontypeid both don't have enough values (these are things like concrete, masonry)
    - This could be interesting later down the road to classify luxury home
- decktypeid, poolsizesum 
- numberofstories doesn't seem like something would be a driver of price 

Turned to 0s 
- hashottuborspa Nan's could be turned to 0s 
- fireplacecnt 
- garagecarcnt can have NaN's converted to 0s 


###  Functions for Nulls 

In [18]:
def nulls_by_row(df):
    '''
    This function takes in a dataframe and returns a dataframe with an overview of how many rows have missing values
    '''
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = round(num_missing / df.shape[1] * 100, 2)
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing


In [19]:
nulls_by_row(df)

,num_cols_missing,percent_cols_missing,num_rows
0,9,19.57,2
1,10,21.74,10
2,11,23.91,23
3,12,26.09,62
4,13,28.26,114
5,14,30.43,416
6,15,32.61,5577
7,16,34.78,10549
8,17,36.96,14385
9,18,39.13,21032


In [20]:
# see what kind of property land use ids we have 
df.propertylandusedesc.value_counts(dropna = False).to_frame()

,propertylandusedesc
Single Family Residential,52319
Condominium,19294
"Duplex (2 Units, Any Combination)",2009
Planned Unit Development,1944
"Quadruplex (4 Units, Any Combination)",727
"Triplex (3 Units, Any Combination)",535
Cluster Home,333
Mobile Home,74
"Manufactured, Modular, Prefabricated Homes",58
Residential General,37


In [21]:
# put in wrangle module
def single_homes(df):
    '''
    Function takes in zillow dataframe and outputs dataframe with only data for single unit homes.
    Single unit home defined as any of the following 
    'Single Family Residential', 'Condominium', 'Townhouse', 'Manufactured, Modular, Prefabricated Homes', 'Mobile Home'
    Home must also have unit count of 1 or NaN
    '''
    # define single home descriptions
    single_homes = ['Single Family Residential', 'Condominium', 'Townhouse', 'Manufactured, Modular, Prefabricated Homes', 'Mobile Home']
    
    # If the property land use description is the in the single homes list keep it
    df = df[df['propertylandusedesc'].isin(single_homes)]
    
    # create mask if unit count is 1 or NaN
    unitcnt_mask = (df['unitcnt'] == 1) | (df['unitcnt'].isnull())
    
    # apply mask to dataframe
    df = df[unitcnt_mask]
    
    return df
    